In [1]:
import pandas as pd
df = pd.read_csv('../../data/raw.csv')
print(f"Example of diagnoses:\n{df['diagnoses'][2]}\n\n")
print(f"Example of operations:\n{df['operations'][2]}\n")

Example of diagnoses:
155516. Cardiac conduit failure;090101. Common arterial trunk;110021. Cardiac arrest


Example of operations:
123610. Replacement of cardiac conduit;123452. Pacemaker system placement: biventricular



In [13]:
#text_fields = ['diagnoses', 'operations']
text_fields = ['diagnoses']

from langchain_huggingface import HuggingFaceEmbeddings
obj_embed_langchain = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from TabuLLM.embed import TextColumnTransformer
obj_embed_tabullm = TextColumnTransformer(obj_embed_langchain, prefix='X', colnames={'diagnoses': 'Diagnoses', 'operations': 'CPB Procedures'})
X_embeddings = obj_embed_tabullm.fit_transform(df[text_fields])

c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
from TabuLLM.cluster import SphericalKMeans
obj_sklearn = SphericalKMeans(n_clusters=10).fit(X_embeddings)
X_labels = obj_sklearn.predict(X_embeddings)
X_str = obj_embed_tabullm.prep_X(df[text_fields])

In [15]:
from langchain_explain import generate_prompt
preamble_1stage, (preamble_2stage_1, preamble_2stage_2), group_payloads = generate_prompt(
    text_list=X_str,
    cluster_labels=X_labels,
    prompt_observations='pediatric CPBs',
    #prompt_texts='patient diagnoses and operations',
    prompt_texts='patient diagnoses',
)

In [16]:
payload_1stage = '\n\n=====\n\n'.join(['Group ' + str(n + 1) + ':\n\n' + payload  for n, payload in enumerate(group_payloads)])

In [6]:
if False:
    from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
    llm = HuggingFaceEndpoint(
        repo_id="HuggingFaceH4/zephyr-7b-beta",
        task="text-generation",
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    )
    obj_chat = ChatHuggingFace(llm=llm)

In [17]:
from langchain_openai import ChatOpenAI

obj_chat = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",
    # base_url="...",
    # organization="...",
    # other params...
)

In [19]:
#print(payload_1stage)
print(preamble_1stage)

The following is a list of 830 pediatric CPBs. Text lines represent patient diagnoses. Data points have been grouped into 10 groups, according to patient diagnoses. Please suggest group labels that are representative of their members and distinct from each other. For each group return the group number, a short description/group label, and a long description of less than 50 words.


In [44]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_core.prompts import ChatPromptTemplate

messages = [
    SystemMessage(content=preamble_1stage),
    HumanMessage(
        content=payload_1stage
    ),
]

prompt = ChatPromptTemplate(messages=messages)


#ai_msg = obj_chat.invoke(messages)

In [49]:
#from pydantic import BaseModel
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser

class GroupLabel(BaseModel):
    group_number: int
    description_short: str
    description_long: str

class MultipleGroupLabels(BaseModel):
    groups: list[GroupLabel]

#parser = JsonOutputParser(pydantic_object=MultipleGroupLabels)
parser = PydanticOutputParser(pydantic_object=MultipleGroupLabels)

In [46]:
chain = prompt | obj_chat | parser

In [48]:
chain.invoke({})

OutputParserException: Invalid json output: Group 1: Cardiomyopathies and Heart Failure
This group includes various types of cardiomyopathies such as dilated, hypertrophic, and restrictive, often associated with heart failure and other cardiac complications.

Group 2: Hypoplastic Left Heart Syndrome and Related Conditions
This group is characterized by hypoplastic left heart syndrome and associated structural heart defects, including aortic arch hypoplasia and coarctation of the aorta.

Group 3: Complex Congenital Heart Defects
This group includes complex congenital heart defects such as pulmonary atresia, discordant VA connections, and cardiac conduit complications, often requiring surgical intervention.

Group 4: Tetralogy of Fallot and Variants
This group is centered around Tetralogy of Fallot and its variants, including cases with pulmonary atresia and absent pulmonary valve syndrome.

Group 5: Aortic and Pulmonary Arterial Abnormalities
This group includes conditions involving aor